In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'case-2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5421570%2F9000200%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240725%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240725T053105Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D90b075afed6b964637b4f63b4c32a9d9bdb27530f5cd1538392616375bce2a73fbcd3aadc890af8dccc4a4ddee74ae7dcc06fdfc6493f992706a1a81f6d72348004ea3019e83708473dc2554a485b27c3064ef39ec793554eb10d77d7077ce5050bff0cb95274b695793c11f2c89ea2d8e5f7b0b33c8d33313b7a9442f82c03ef4940c3acb06deb3ba09c353455d8461a2a8acd6ce0eac8a09c9932f38caae78051f428d1d25e7f5dcd90c2ea31bf824459f870bd3a0708ef9c6361e0c08d4b731852d16f729e01306a72c37374befa6ec4c13ddd2be79534d53d11becfc11a6feddf596b1f7985929ce8224151b52c36ff96ce0e79c4506d94809cedfd5959e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
dataset = pd.read_csv("/kaggle/input/case-2/covid19_Confirmed_dataset.csv")
dataset.head()


In [ ]:
dataset.shape

In [ ]:
df  = dataset.drop(["Lat","Long"], axis= 1, inplace = True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
corona_dataset_aggregated = dataset.groupby("Country/Region").sum()
corona_dataset_aggregated

In [ ]:
corona_dataset_aggregated

In [ ]:

corona_dataset_aggregated.loc["Afghanistan"].plot()
corona_dataset_aggregated.loc["India"].plot()
#corana_dataset_aggregated.loc["Australia"].plot()

In [ ]:
country = list(corona_dataset_aggregated.index)

In [ ]:

corona_dataset_aggregated.loc["India"].diff().max()

In [ ]:
corona_dataset_aggregated.loc["Spain"].diff().max()

In [ ]:
corona_dataset_aggregated.loc["Andorra"].diff()

In [ ]:
#orana_dataset_aggregated

In [ ]:
corona_dataset_aggregated.dtypes

In [ ]:

corona_dataset_aggregated = corona_dataset_aggregated.apply(pd.to_numeric, errors='coerce')
countries = list(corona_dataset_aggregated.index)
max_infection_rates = []

for c in countries:

    max_infection_rate = corona_dataset_aggregated.loc[c].diff().max()
    max_infection_rates.append(max_infection_rate)

corona_dataset_aggregated["Max_infection_rates"] = max_infection_rates

In [ ]:
corona_dataset_aggregated

In [ ]:
corona_data = pd.DataFrame(corona_dataset_aggregated["Max_infection_rates"])

In [ ]:
corona_data

In [ ]:
#import datasheet 2nd

In [ ]:
happiness_report = pd.read_csv("/kaggle/input/case-2/worldwide_happiness_report.csv")

In [ ]:
happiness_report

In [ ]:
useless_cols = ["Overall rank","Score","Generosity","Perceptions of corruption"]

In [ ]:
happiness_report.drop(useless_cols,axis = 1,inplace = True)

In [ ]:
happiness_report.head()

In [ ]:
happiness_report.set_index("Country or region", inplace = True)

In [ ]:
happiness_report

In [ ]:
corona_data.shape

In [ ]:
happiness_report.shape

In [ ]:
data = corona_data.join(happiness_report, how = "inner")

In [ ]:
data

In [ ]:
data.corr()

In [ ]:
x= data["GDP per capita"]
y= data["Healthy life expectancy"]
z=data["Social support"]
y.transformed = np.log(y)
x.transformed = x/100
px.scatter_3d(x=x.transformed,y=y.transformed,z=z)

In [ ]:
sns.regplot(x=x.transformed,y=y.transformed)

In [ ]:
sns.scatterplot(x=x.transformed,y=y.transformed)

In [ ]:
x= data["GDP per capita"]
y= data["Max_infection_rates"]
y.transformed = np.log(y)
x.transformed = x

sns.scatterplot(x=x.transformed,y=y.transformed)

In [ ]:
sns.regplot(x=x.transformed,y=y.transformed)

In [ ]:
x= data["Social support"]
y =data["Max_infection_rates"]
y.transformed = np.log(y)
x.transformed = x

sns.scatterplot(x=x.transformed,y=y.transformed)

In [ ]:
sns.regplot(x=x.transformed,y=y.transformed)

In [ ]:
x= data["Healthy life expectancy"]
y =data["Max_infection_rates"]
y.transformed = np.log(y)
x.transformed = x

sns.scatterplot(x=x.transformed,y=y.transformed)

In [ ]:
sns.regplot(x=x.transformed,y=y.transformed)

In [ ]:
x= data["Freedom to make life choices"]
y =data["Max_infection_rates"]
y.transformed = np.log(y)
x.transformed = x

sns.scatterplot(x=x.transformed,y=y.transformed)

In [ ]:
sns.regplot(x=x.transformed,y=y.transformed)